# Data Preprocessing
First import the data.

In [1]:
import numpy as np
X_test = np.load("project/X_test.npy")
y_test = np.load("project/y_test.npy")
person_train_valid = np.load("project/person_train_valid.npy")
X_train_valid = np.load("project/X_train_valid.npy")
y_train_valid = np.load("project/y_train_valid.npy")
person_test = np.load("project/person_test.npy")

In [2]:
# Ignore EOG data from last 3 of 25 electrodes
# Transpose data so that the second dimension is timesteps
X_test = X_test[:, :-3, :]
X_test = X_test.transpose([0, 2, 1])
X_train_valid = X_train_valid[:, :-3, :]
X_train_valid = X_train_valid.transpose([0, 2, 1])

In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 1000, 22)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
from sklearn import preprocessing

In [5]:
np.random.seed(0)

In [6]:
num_trials = X_train_valid.shape[0]
num_timesteps = X_train_valid.shape[1]
num_features = X_train_valid.shape[2]

In [7]:
# Turn labels into categories, i.e. y values of 0-3
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train_valid)
y_train_valid_classes = le.transform(y_train_valid)

In [26]:
rand_idx = np.random.permutation(num_trials)
rand_idx 

array([1593,  463,  901, ..., 1091,  880, 1750])

In [9]:
# Create randomized train and validation dataset split
split = int(num_trials * 0.80)
X_train = X_train_valid[rand_idx][:split]
y_train = y_train_valid_classes[rand_idx][:split]
person_train = person_train_valid[rand_idx][:split]

X_valid = X_train_valid[rand_idx][split:]
y_valid = y_train_valid_classes[rand_idx][split:]
person_valid = person_train_valid[rand_idx][split:]

In [33]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from keras import optimizers
from keras.layers.normalization import BatchNormalization

In [34]:
checkpoint_path = 'checkpoints/EEG_prediction.ckpt'
import keras.callbacks

# Create checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True, 
                                                 monitor='val_loss',
                                                 save_best_only=True, 
                                                 mode='auto',
                                                 verbose=1)

In [35]:
# Build model architecture
model = Sequential()

model.add(GRU(256, dropout=0.3, input_shape=(num_timesteps, num_features), return_sequences=True))
model.add(BatchNormalization())

model.add(GRU(128, dropout=0.3))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(4, activation='softmax'))
#model.load_weights(checkpoint_path)

In [ ]:
# Fit model on training set, validate with validation data
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200, batch_size=64, verbose=1, callbacks = [cp_callback])

Train on 1692 samples, validate on 423 samples
Epoch 1/200
1692/1692 [==============================] - 174s 103ms/step - loss: 1.8630 - acc: 0.2441 - val_loss: 1.6254 - val_acc: 0.2435

Epoch 00001: val_loss improved from inf to 1.62544, saving model to checkpoints/EEG_prediction.ckpt
Epoch 2/200
1692/1692 [==============================] - 175s 103ms/step - loss: 1.6733 - acc: 0.2683 - val_loss: 1.5072 - val_acc: 0.2364

Epoch 00002: val_loss improved from 1.62544 to 1.50717, saving model to checkpoints/EEG_prediction.ckpt
Epoch 3/200
1692/1692 [==============================] - 168s 99ms/step - loss: 1.6398 - acc: 0.2547 - val_loss: 1.5243 - val_acc: 0.2317

Epoch 00003: val_loss did not improve from 1.50717
Epoch 4/200
1692/1692 [==============================] - 196s 116ms/step - loss: 1.5664 - acc: 0.2766 - val_loss: 1.5132 - val_acc: 0.2459

Epoch 00004: val_loss did not improve from 1.50717
Epoch 5/200
1692/1692 [==============================] - 194s 115ms/step - loss: 1.5576 